In [1]:
pip install datasets

  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW

In [2]:
cbt_dataset = load_dataset('cbt','CN')

Found cached dataset cbt (C:/Users/34941/.cache/huggingface/datasets/cbt/CN/1.1.0/dc4451a8a4b50cebb78fdb19fa9f964b27fcdcef915467b8b7055a3a8d8cef7b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train_set = cbt_dataset['train']
val_set = cbt_dataset['validation']
test_set = cbt_dataset['test']

In [5]:
train_set

Dataset({
    features: ['sentences', 'question', 'answer', 'options'],
    num_rows: 120769
})

In [4]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

C:\Users\34941\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\34941\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
context, question, answer, options = train_set[0]['sentences'], train_set[0]['question'], train_set[0]['answer'],train_set[0]['options']

In [22]:
input_text = f"{context} {question} {' '.join(options)}"
input_ids = tokenizer.encode(input_text, add_special_tokens=True)
labels = [0] * len(input_ids)
answer_id = tokenizer.encode(answer)[0]

In [23]:
tokenizer.encode(answer)

[4188, 268]

In [19]:
        context, question, answer, options = train_set[0]['sentence'],


        labels = [0] * len(input_ids)
        answer_id = self.tokenizer.encode(answer)[0]
        labels[-len(options) + options.index(answer)] = answer_id
        return torch.tensor(input_ids), torch.tensor(labels)

In [ ]:
# Define model and optimizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define training loop
def train(model, train_dataloader, optimizer, scheduler=None, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for input_ids, labels in train_dataloader:
            input_ids, labels = input_ids.to(device), labels.to(device)
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        print(f"Epoch {epoch+1} Loss: {total_loss/len(train_dataloader)}")

# Load dataset and create dataloader
dataset = ReadingComprehensionDataset('data.txt')
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Fine-tune model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
num_training_steps = len(train_dataloader) * 3
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)
train(model, train_dataloader, optimizer, scheduler=scheduler, num_epochs=3)